In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from nn_model import random_search

import config

from utils import graph_nn_model

import os

import pandas as pd

import torch

from datetime import datetime

Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)


In [3]:
search_obj = random_search(config.DEVICE)
search_obj.load('./randomsearch/2023_10_17_19_32_47.p')
best_model = search_obj.best_model
best_model= best_model.to(config.DEVICE)
print(best_model)

linear_model(
  (layers): ModuleList(
    (0): Sequential(
      (0): Linear(in_features=600, out_features=512, bias=True)
      (1): GELU(approximate='none')
      (2): Identity()
    )
    (1): Sequential(
      (0): Linear(in_features=512, out_features=128, bias=True)
      (1): GELU(approximate='none')
      (2): Identity()
    )
    (2): Sequential(
      (0): Linear(in_features=128, out_features=128, bias=True)
      (1): GELU(approximate='none')
      (2): Identity()
    )
    (3): Sequential(
      (0): Linear(in_features=128, out_features=64, bias=True)
      (1): GELU(approximate='none')
      (2): Identity()
    )
    (4): Linear(in_features=64, out_features=1, bias=True)
  )
)


In [4]:
drugname = '5-Fluorouracil'

select_drugname = True

In [5]:
df = pd.read_csv("./data/Test_data.csv")
df.columns = pd.read_csv("./data/Training_data.csv").columns



In [6]:

df.rename({"CELL_LINE_NAME":"Cell Line Name", "DRUG_NAME":"Drug Name"}, axis = 1, inplace = True)


In [7]:
if select_drugname:

    df = df[df['Drug Name'] == drugname]

In [8]:
test_x = df.drop(['Drug Name', 'Cell Line Name',"LN_IC50"], axis = 1)
test_y = df['LN_IC50']

In [9]:
test_x_tensor = torch.tensor(test_x.values).to(torch.float32).to(config.DEVICE)
test_y_tensor = torch.tensor(test_y.values).to(torch.float32).to(config.DEVICE)

In [10]:
#not using

'''

#graphing the model 

os.makedirs("./graphs/dnn_image",exist_ok=True)

graph_nn_model(best_model.to(config.DEVICE),sample_row.to(config.DEVICE),'./graphs/dnn_image/model_image')

'''

'\n\n#graphing the model \n\nos.makedirs("./graphs/dnn_image",exist_ok=True)\n\ngraph_nn_model(best_model.to(config.DEVICE),sample_row.to(config.DEVICE),\'./graphs/dnn_image/model_image\')\n\n'

## Model predictions

In [11]:
# model predictions

os.makedirs("./predictions/dnn", exist_ok=True)

In [12]:
predictions = best_model(test_x_tensor)

In [13]:
result_df = pd.DataFrame(predictions.cpu().detach().numpy())

result_df.columns = ['predictions']

result_df['actual'] = df['LN_IC50'].values

In [14]:




prediction_name = "gene_predict_IC50"

tm = datetime.now()


dir_save_all_model = f"{prediction_name}_{tm.year}_{tm.month}_{tm.day}_{tm.strftime('%H_%M_%S')}"

result_df.to_csv(f"./predictions/dnn/{dir_save_all_model}.csv", index=False)
    

## Feature importance graph

In [15]:
from utils import calculate_dnn_feature_importance

In [16]:
df_feature_importance = calculate_dnn_feature_importance(test_x,test_y, best_model, 
                                                         do_permutation= True,
                                                         do_ablation=False,
                                                         do_shap= False)

In [17]:
os.makedirs("./feature_importance/dnn", exist_ok=True)

In [18]:
tm = datetime.now()

df_feature_importance.to_csv(f"./feature_importance/dnn/{tm.year}_{tm.month}_{tm.day}_{tm.strftime('%H_%M_%S')}.csv", index = False)

